## Clustering using SparkML


<p style='color: red'>The purpose of this lab is to show you how to use SparkML to cluster data.


## __Table of Contents__
<ol>
  <li>
    <a href="#Objectives">Objectives
    </a>
  </li>
  <li>
    <a href="#Datasets">Datasets
    </a>
  </li>
  <li>
    <a href="#Setup">Setup
    </a>
    <ol>
      <li>
        <a href="#Installing-Required-Libraries">Installing Required Libraries
        </a>
      </li>
      <li>
        <a href="#Importing-Required-Libraries">Importing Required Libraries
        </a>
      </li>
    </ol>
  </li>
  <li>
    <a href="#Examples">Examples
    </a>
    <ol>
      <li>
        <a href="#Task-1---Create-a-spark-session">Task 1 - Create a spark session
        </a>
      </li>
      <li>
        <a href="#Task-2---Load-the-data-in-a-csv-file-into-a-dataframe">Task 2 - Load the data in a csv file into a dataframe
        </a>
      </li>
      <li>
        <a href="#Task-3---Create-a-feature-vector">Task 3 - Create a feature vector
        </a>
      </li>
      <li>
        <a href="#Task-4---Create-a-clustering-model">Task 4 - Create a clustering model
        </a>
      </li>
      <li>
        <a href="#Task-5---Print-Cluster-Centers">Task 5 - Print Cluster Centers
        </a>
      </li>
    </ol>
  </li>
  <li>
    <a href="#Exercises">Exercises
    </a>
  </li>
  <ol>
    <li>
      <a href="#Exercise-1---Create-a-spark-session">Exercise 1 - Create a spark session
      </a>
    </li>
    <li>
      <a href="#Exercise-2---Load-the-data-in-a-csv-file-into-a-dataframe">Exercise 2 - Load the data in a csv file into a dataframe
      </a>
    </li>
    <li>
      <a href="#Exercise-3---Create-a-feature-vector">Exercise 3 - Create a feature vector
      </a>
    </li>
    <li>
      <a href="#Exercise-4---Create-a-clustering-model">Exercise 4 - Create a clustering model
      </a>
    </li>
    <li>
      <a href="#Exercise-5---Print-Cluster-Centers">Exercise 5 - Print Cluster Centers
      </a>
    </li>
  </ol>
</ol>


## Objectives

After completing this lab you will be able to:

 - Use PySpark to connect to a spark cluster.
 - Create a spark session.
 - Read a csv file into a data frame.
 - Use KMeans algorithm to cluster the data
 - Stop the spark session





## Datasets

In this lab you will be using dataset(s):

 - Modified version of Wholesale customers dataset. Original dataset available at https://archive.ics.uci.edu/ml/datasets/Wholesale+customers
 - Seeds dataset. Available at https://archive.ics.uci.edu/ml/datasets/seeds


----


## Setup


For this lab, we will be using the following libraries:

*   [`PySpark`](https://spark.apache.org/docs/latest/api/python/index.html?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMSkillsNetworkBD0231ENCoursera2789-2023-01-01) for connecting to the Spark Cluster


### Installing Required Libraries

Spark Cluster is pre-installed in the Skills Network Labs environment. However, you need libraries like pyspark and findspark to connect to this cluster.

If you wish to download this jupyter notebook and run on your local computer, follow the instructions mentioned <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/labs/Connecting_to_spark_cluster_using_Skills_Network_labs.ipynb">here.</a>



The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [61]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [62]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

#import functions/Classes for sparkml

from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

from pyspark.sql import SparkSession


## Examples


## Task 1 - Create a spark session


In [63]:
#Create SparkSession
#Ignore any warnings by SparkSession command

spark = SparkSession.builder.appName("Clustering using SparkML").getOrCreate()

## Task 2 - Load the data in a csv file into a dataframe


Download the data file


In [64]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/customers.csv|


/bin/bash: -c: line 2: syntax error: unexpected end of file


Load the dataset into the spark dataframe


In [66]:
# using the spark.read.csv function we load the data into a dataframe.
# the header = True mentions that there is a header row in out csv file
# the inferSchema = True, tells spark to automatically find out the data types of the columns.

# Load customers dataset
customer_data = spark.read.csv("/content/customers.csv", header=True, inferSchema=True)


Print the schema of the dataset


In [36]:
# Each row in this dataset is about a customer. The columns indicate the orders placed
# by a customer for Fresh_food, Milk, Grocery and Frozen_Food

In [67]:
customer_data.printSchema()

root
 |-- Fresh_Food: integer (nullable = true)
 |-- Milk: integer (nullable = true)
 |-- Grocery: integer (nullable = true)
 |-- Frozen_Food: integer (nullable = true)



Show top 5 rows from the dataset


In [68]:
customer_data.show(n=5, truncate=False)

+----------+----+-------+-----------+
|Fresh_Food|Milk|Grocery|Frozen_Food|
+----------+----+-------+-----------+
|12669     |9656|7561   |214        |
|7057      |9810|9568   |1762       |
|6353      |8808|7684   |2405       |
|13265     |1196|4221   |6404       |
|22615     |5410|7198   |3915       |
+----------+----+-------+-----------+
only showing top 5 rows



## Task 3 - Create a feature vector


In [69]:
# Assemble the features into a single vector column
feature_cols = ['Fresh_Food', 'Milk', 'Grocery', 'Frozen_Food']
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
customer_transformed_data = assembler.transform(customer_data)



You must tell the KMeans algorithm how many clusters to create out of your data


In [70]:
number_of_clusters = 3

## Task 4 - Create a clustering model


Create a KMeans clustering model


In [71]:
kmeans = KMeans(k = number_of_clusters)


Train/Fit the model on the dataset<br>


In [72]:
model = kmeans.fit(customer_transformed_data)


## Task 5 - Print Cluster Details


Your model is now trained. Time to evaluate the model.


In [73]:
# Make predictions on the dataset
predictions = model.transform(customer_transformed_data)

In [74]:
# Display the results
predictions.show(5)

+----------+----+-------+-----------+--------------------+----------+
|Fresh_Food|Milk|Grocery|Frozen_Food|            features|prediction|
+----------+----+-------+-----------+--------------------+----------+
|     12669|9656|   7561|        214|[12669.0,9656.0,7...|         0|
|      7057|9810|   9568|       1762|[7057.0,9810.0,95...|         0|
|      6353|8808|   7684|       2405|[6353.0,8808.0,76...|         0|
|     13265|1196|   4221|       6404|[13265.0,1196.0,4...|         0|
|     22615|5410|   7198|       3915|[22615.0,5410.0,7...|         1|
+----------+----+-------+-----------+--------------------+----------+
only showing top 5 rows



Display how many customers are there in each cluster.


In [75]:
predictions.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   62|
|         2|   25|
|         0|  353|
+----------+-----+



In [76]:
#stop spark session
spark.stop()

# Exercises


### Exercise 1 - Create a spark session


Create SparkSession with appname "Seed Clustering"


In [77]:
spark = #TODO

SyntaxError: invalid syntax (<ipython-input-77-d68a6628e27f>, line 1)

<details>
    <summary>Click here for a Hint</summary>
    
Use the SparkSession.builder

</details>


<details>
    <summary>Click here for Solution</summary>

```python
spark = SparkSession.builder.appName("Seed Clustering").getOrCreate()
```

</details>


### Exercise 2 - Load the data in a csv file into a dataframe


In [49]:
#download seed dataset
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/seeds.csv


--2024-11-19 05:58:52--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/seeds.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8973 (8.8K) [text/csv]
Saving to: ‘seeds.csv’

seeds.csv           100%[===================>]   8.76K  --.-KB/s    in 0s      

2024-11-19 05:58:53 (106 MB/s) - ‘seeds.csv’ saved [8973/8973]



Load the seed dataset


In [56]:

seed_data =  #TODO


SyntaxError: invalid syntax (<ipython-input-56-fc568baa7a66>, line 1)

<details>
    <summary>Click here for a Hint</summary>
    
Use the spark.read.csv

</details>


<details>
    <summary>Click here for Solution</summary>

```python
seed_data = spark.read.csv("seeds.csv", header=True, inferSchema=True)
```

</details>


Print the schema of the dataset


In [57]:
seed_data.printSchema()

NameError: name 'seed_data' is not defined

Show top 5 rows of the data set


In [52]:
seed_data.show(n=5, truncate=False, vertical=True)

NameError: name 'seed_data' is not defined

### Exercise 3 - Create a feature vector


Assemble all columns into a single vector


In [53]:
feature_cols =  #TODO
assembler =  #TODO
seed_transformed_data =  #TODO


SyntaxError: invalid syntax (<ipython-input-53-f08ddf7cf0f7>, line 1)

<details>
    <summary>Click here for a Hint</summary>
    
Refer to task - 3
</details>


<details>
    <summary>Click here for Solution</summary>

```python
feature_cols = ['area',
 'perimeter',
 'compactness',
 'length of kernel',
 'width of kernel',
 'asymmetry coefficient',
 'length of kernel groove']

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
seed_transformed_data = assembler.transform(seed_data)

```

</details>


### Exercise 4 - Create a clustering model


Create 7 clusters


In [55]:
number_of_clusters =  #TODO
kmeans =  #TODO
model =  #TODO

SyntaxError: invalid syntax (<ipython-input-55-7ece2b098cc6>, line 1)

<details>
    <summary>Click here for a Hint</summary>
    
use the kmeans.fit() method
</details>


<details>
    <summary>Click here for Solution</summary>

```python
number_of_clusters = 3
kmeans = KMeans(k = number_of_clusters)
model = kmeans.fit(seed_transformed_data)

```

</details>


### Exercise 5 - Print Cluster Details


In [ ]:
predictions =  #TODO

<details>
    <summary>Click here for a Hint</summary>
    
use the transform() method
</details>


<details>
    <summary>Click here for Solution</summary>

```python
predictions = model.transform(seed_transformed_data)
```

</details>


In [ ]:
predictions.show(n=5, truncate=False, vertical=True)

In [ ]:
predictions.groupBy('prediction').count().show()

In [ ]:
#stop spark session
spark.stop()

<!--## Change Log
-->


<!--
|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2023-05-01|0.1|Ramesh Sannareddy|Initial Version Created|
-->
